<a href="https://colab.research.google.com/github/Gaabiin5/Hugging_Face_Formation/blob/main/Code_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Agents That Use Code

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

Alfred is planning a party at the Wayne family mansion and needs your help to ensure everything goes smoothly. To assist him, we'll apply what we've learned about how a multi-step `CodeAgent` operates.


## Let's install the dependencies and login to our HF account to access the Inference API

If you haven't installed `smolagents` yet, you can do so by running the following command:

In [1]:
import sys
print(sys.executable)


/home/gan/venv/bin/python


In [ ]:
%pip install smolagents -U

%pip install 'smolagents[litellm]' -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.8/514.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 42.2 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.3 MB/s eta 0:00:0031m41.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 6.1 MB/s eta 0:00:00


In [ ]:
%pip install duckduckgo-search -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 39.1 MB/s eta 0:00:0031m46.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.1 MB/s eta 0:00:00m eta 0:00:010:01


Let's also login to the Hugging Face Hub to have access to the Inference API.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Pour eviter les soucis de credit. Utiliser un serveur local ollama avec les modeles LLM. Colabxterm permet d'avoir un terminal qui tourne en meme temps que les scripts collabs.

Les commandes sont

ollama pull qwen2:7b (pour charger le modele et pouvoir l'utiliser ensuite)
ollama serve (pour lancer le serveur)
sudo lsof -i :11434 (pour verifier qui utilise le truc local si la commande precedente ne marche pas)

In [ ]:
# !pip install colab-xterm
# %load_ext colabxterm


The colabxterm extension is already loaded. To reload it, use:
  %reload_ext colabxterm


In [ ]:
# !curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
[sudo] password for gan: 
sudo: a password is required
^C


In [ ]:
# %xterm # Run ollama serve in the xterm

UsageError: Line magic function `%xterm` not found.


In [ ]:
# !ollama pull qwen2:7b

## Selecting a Playlist for the Party Using `smolagents`

An important part of a successful party is the music. Alfred needs some help selecting the playlist. Luckily, `smolagents` has got us covered! We can build an agent capable of searching the web using DuckDuckGo. To give the agent access to this tool, we include it in the tool list when creating the agent.

For the model, we'll rely on `InferenceClientModel`, which provides access to Hugging Face's [Inference API](https://huggingface.co/docs/api-inference/index). The default model is `"Qwen/Qwen2.5-Coder-32B-Instruct"`, which is performant and available for fast inference, but you can select any compatible model from the Hub.

Running an agent is quite straightforward:

In [7]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel

from smolagents import LiteLLMModel

model = LiteLLMModel(
        model_id="ollama_chat/qwen2:7b",  # Or try other Ollama-supported models
        api_base="http://127.0.0.1:11434",  # Default Ollama local server
        num_ctx=8192,
)

agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=model)

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for the best music recommendations for a party at the Wayne's mansion.                                   │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2:7b ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Perform web search for 'best music for party' and filter results for 'Wayne's mansion'                         
  search_results = web_search(query='best music for party')                                                        
                                                                                                                   
  # Iterate through the search results to find mentions of Wayne's mansion or suitable party vibes                 
  party_music_recommendations = []                                                                                 
  for result in search_results:                                                                                    
      if ('Wayne\'s mansion' in result) or ('energetic', 'dance', 'house') in result.lower():                      
          # Assuming 'result' contains song names, artists and some description                                    
          song_name = extract_from_text(result, "Song: ", " - ")  # Extracting song name using simple text         
  processing                                                                                                       
          artist = extract_from_text(result, "By ", " |")  # Extracting artist's name                              
                                                                                                                   
          if song_name and artist:                                                                                 
              party_music_recommendations.append(f"{song_name} by {artist}")                                       
                                                                                                                   
  # Show the top recommendations we've found for a party at Wayne's mansion                                        
  print(party_music_recommendations)                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'for result in search_results:
    if ('Wayne\'s mansion' in result) or ('energetic', 'dance', 'house') in result.lower():
        # Assuming 'result' contains song names, artists and some description
        song_name = extract_from_text(result, "Song: ", " - ")  # Extracting song name using simple text processing
        artist = extract_from_text(result, "By ", " |")  # Extracting artist's name
        
        if song_name and artist:
            party_music_recommendations.append(f"{song_name} by {artist}")' due to: TypeError: 'in <string>' 
requires string as left operand, not tuple

[Step 1: Duration 7.36 seconds| Input tokens: 2,062 | Output tokens: 246]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Display the final result indicating that we couldn't find suitable party music for Wayne's mansion             
  final_answer('Unfortunately, no suitable party music found related to "Wayne\'s mansion" and energetic dance     
  house.')                                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Unfortunately, no suitable party music found related to "Wayne's mansion" and energetic dance 
house.

[Step 5: Duration 4.16 seconds| Input tokens: 16,709 | Output tokens: 1,029]

'Unfortunately, no suitable party music found related to "Wayne\'s mansion" and energetic dance house.'

When you run this example, the output will **display a trace of the workflow steps being executed**. It will also print the corresponding Python code with the message:

```python
 ─ Executing parsed code: ────────────────────────────────────────────────────────────────────────────────────────
  results = web_search(query="best music for a Batman party")                                                      
  print(results)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────
```

After a few steps, you'll see the generated playlist that Alfred can use for the party! 🎵

## Using a Custom Tool to Prepare the Menu

Now that we have selected a playlist, we need to organize the menu for the guests. Again, Alfred can take advantage of `smolagents` to do so. Here, we use the `@tool` decorator to define a custom function that acts as a tool. We'll cover tool creation in more detail later, so for now, we can simply run the code.

As you can see in the example below, we will create a tool using `@tool` decorator and include it in the `tools` list.

In [18]:
from smolagents import CodeAgent, tool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion (str): The type of occasion for the party. Allowed values are:
                        - "casual": Menu for casual party.
                        - "formal": Menu for formal party.
                        - "superhero": Menu for superhero party.
                        - "custom": Custom menu.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

agent = CodeAgent(tools=[suggest_menu], model=model)

agent.run("Prepare a formal menu for the party.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Prepare a formal menu for the party.                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2:7b ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  menu = suggest_menu(occasion="formal")                                                                           
  print(menu)                                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
3-course dinner with wine and dessert.

Out: None

[Step 1: Duration 6.74 seconds| Input tokens: 2,084 | Output tokens: 67]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("3-course dinner with wine and dessert")                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 3-course dinner with wine and dessert

[Step 2: Duration 1.80 seconds| Input tokens: 4,317 | Output tokens: 124]

'3-course dinner with wine and dessert'

The agent will run for a few steps until finding the answer.

The menu is ready! 🥗

## Using Python Imports Inside the Agent

We have the playlist and menu ready, but we need to check one more crucial detail: preparation time!

Alfred needs to calculate when everything would be ready if he started preparing now, in case they need assistance from other superheroes.

`smolagents` specializes in agents that write and execute Python code snippets, offering sandboxed execution for security. It supports both open-source and proprietary language models, making it adaptable to various development environments.

**Code execution has strict security measures** - imports outside a predefined safe list are blocked by default. However, you can authorize additional imports by passing them as strings in `additional_authorized_imports`.
For more details on secure code execution, see the official [guide](https://huggingface.co/docs/smolagents/tutorials/secure_code_execution).

When creating the agent, we ill use `additional_authorized_imports` to allow for importing the `datetime` module.

In [19]:
from smolagents import CodeAgent, InferenceClientModel, LiteLLMModel
import numpy as np
import time
import datetime

model = LiteLLMModel(
        model_id="ollama_chat/qwen2:7b",  # Or try other Ollama-supported models
        api_base="http://127.0.0.1:11434",  # Default Ollama local server
        num_ctx=8192,
)

agent = CodeAgent(tools=[], model=model, additional_authorized_imports=['datetime'])

agent.run(
    """
    Alfred needs to prepare for the party. Here are the tasks:
    1. Prepare the drinks - 30 minutes
    2. Decorate the mansion - 60 minutes
    3. Set up the menu - 45 minutes
    3. Prepare the music and playlist - 45 minutes

    If we start right now, at what time will the party be ready?
    """
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Alfred needs to prepare for the party. Here are the tasks:                                                      │
│     1. Prepare the drinks - 30 minutes                                                                          │
│     2. Decorate the mansion - 60 minutes                                                                        │
│     3. Set up the menu - 45 minutes                                                                             │
│     3. Prepare the music and playlist - 45 minutes                                                              │
│                                                                                                                 │
│     If we start right now, at what time will the party be ready?                                                │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2:7b ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  total_time = 30 + 60 + 45 + 45                                                                                   
  print(f"The total time needed for preparations is {total_time} minutes.")                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The total time needed for preparations is 180 minutes.

Out: None

[Step 1: Duration 2.77 seconds| Input tokens: 2,083 | Output tokens: 75]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The party will be ready at {}".format(party_ready_time.strftime('%Y-%m-%d %H:%M')))                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The party will be ready at 2025-06-17 18:39

[Step 3: Duration 3.38 seconds| Input tokens: 6,852 | Output tokens: 278]

'The party will be ready at 2025-06-17 18:39'

These examples are just the beginning of what you can do with code agents, and we're already starting to see their utility for preparing the party.
You can learn more about how to build code agents in the [smolagents documentation](https://huggingface.co/docs/smolagents).

`smolagents` specializes in agents that write and execute Python code snippets, offering sandboxed execution for security. It supports both open-source and proprietary language models, making it adaptable to various development environments.

## Sharing Our Custom Party Preparator Agent to the Hub

Wouldn't it be **amazing to share our very own Alfred agent with the community**? By doing so, anyone can easily download and use the agent directly from the Hub, bringing the ultimate party planner of Gotham to their fingertips! Let's make it happen! 🎉

The `smolagents` library makes this possible by allowing you to share a complete agent with the community and download others for immediate use. It's as simple as the following:


In [10]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel, VisitWebpageTool, FinalAnswerTool, Tool, tool

model = LiteLLMModel(
        model_id="ollama_chat/qwen2:7b",  # Or try other Ollama-supported models
        api_base="http://127.0.0.1:11434",  # Default Ollama local server
        num_ctx=8192,
)

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.

    Args:
        query: A search term for finding catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }

    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)

    return best_service

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""

    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic gotham').",
        }
    }

    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }

        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic gotham'.")


# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool()
        ],
    model=model,
    max_steps=10,
    verbosity_level=2
)

agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme        │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2:7b ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To find the best playlist I need to search for suitable songs related to villain masquerade theme.        
Code:                                                                                                              
```py                                                                                                              
playlist_suggestions = superhero_party_theme_generator('villain masquerade')                                       
print("Suggested playlist:", playlist_suggestions)                                                                 
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  playlist_suggestions = superhero_party_theme_generator('villain masquerade')                                     
  print("Suggested playlist:", playlist_suggestions)                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Suggested playlist: Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.

Out: None

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: The suggested playlist is ready.                                                                          
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
final_answer(music_playlist)                                                                                       
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(music_playlist)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: ['"The Joker"', '"Twins of Evil"', '"Poison"', '"Monster Mash"', '"Wipeout"']

[Step 7: Duration 1.44 seconds| Input tokens: 19,839 | Output tokens: 409]

['"The Joker"', '"Twins of Evil"', '"Poison"', '"Monster Mash"', '"Wipeout"']

In [ ]:
!git config --global credential.helper store
!git clone https://huggingface.co/Gaabin/test

Cloning into 'test'...
Username for 'https://huggingface.co': ^C


In [15]:
from huggingface_hub import notebook_login

notebook_login()

In [20]:
agent.push_to_hub('Gaabin/AlfredAgent')

No files have been modified since last commit. Skipping to prevent empty commit.


This agent has step_callbacks: they will be ignored by this method. 1

For security reasons, we do not export the `api_key` attribute of your model. Please export it manually.


CommitInfo(commit_url='https://huggingface.co/spaces/Gaabin/AlfredAgent/commit/47a4629d3745d6e45ad7c304bdbc23ef53c3c30e', commit_message='Upload agent', commit_description='', oid='47a4629d3745d6e45ad7c304bdbc23ef53c3c30e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/spaces/Gaabin/AlfredAgent', endpoint='https://huggingface.co', repo_type='space', repo_id='Gaabin/AlfredAgent'), pr_revision=None, pr_num=None)

To download the agent again, use the code below:

In [23]:
import random

In [24]:
agent = CodeAgent(tools=[], model=model)
alfred_agent = agent.from_hub('Gaabin/AlfredAgent', trust_remote_code=True)

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

In [26]:
alfred_agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme        │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2:7b ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  playlist = []                                                                                                    
                                                                                                                   
  # Web search for villains' theme song                                                                            
  response_villain_songs = web_search('best villain theme songs')                                                  
  for result in response_villain_songs:                                                                            
      # Extract the top 5 results that seem most fitting as a playlist item                                        
      if len(playlist) < 5:                                                                                        
          playlist.append(result)                                                                                  
                                                                                                                   
  # Check what we've found to ensure it's relevant and interesting                                                 
  print(playlist)                                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'response_villain_songs = web_search('best villain theme songs')' due to: 
InterpreterError: Forbidden function evaluation: 'web_search' is not among the explicitly allowed tools or 
defined/imported in the preceding code

[Step 1: Duration 3.85 seconds| Input tokens: 2,023 | Output tokens: 145]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(playlist)                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: ['Twilight Zone Main Theme', "The Dark Knight: Joker's Hilarious Murder Theme", 'The Godfather 
II - Morricone Medley', "Pirates of the Caribbean: He's A Pirate!", 'Transformers (1984 Animated Series) - Rock and
Roll All Nite']

[Step 3: Duration 1.50 seconds| Input tokens: 7,226 | Output tokens: 335]

['Twilight Zone Main Theme',
 "The Dark Knight: Joker's Hilarious Murder Theme",
 'The Godfather II - Morricone Medley',
 "Pirates of the Caribbean: He's A Pirate!",
 'Transformers (1984 Animated Series) - Rock and Roll All Nite']

What's also exciting is that shared agents are directly available as Hugging Face Spaces, allowing you to interact with them in real-time. You can explore other agents [here](https://huggingface.co/spaces/davidberenstein1957/smolagents-and-tools).

For example, the _AlfredAgent_ is available [here](https://huggingface.co/spaces/sergiopaniego/AlfredAgent).

### Inspecting Our Party Preparator Agent with OpenTelemetry and Langfuse 📡

Full trace can be found [here](https://cloud.langfuse.com/project/cm7bq0abj025rad078ak3luwi/traces/995fc019255528e4f48cf6770b0ce27b?timestamp=2025-02-19T10%3A28%3A36.929Z).

As Alfred fine-tunes the Party Preparator Agent, he's growing weary of debugging its runs. Agents, by nature, are unpredictable and difficult to inspect. But since he aims to build the ultimate Party Preparator Agent and deploy it in production, he needs robust traceability for future monitoring and analysis.  

Once again, `smolagents` comes to the rescue! It embraces the [OpenTelemetry](https://opentelemetry.io/) standard for instrumenting agent runs, allowing seamless inspection and logging. With the help of [Langfuse](https://langfuse.com/) and the `SmolagentsInstrumentor`, Alfred can easily track and analyze his agent’s behavior.  

Setting it up is straightforward!  

First, we need to install the necessary dependencies:  

In [ ]:
%pip install smolagents[telemetry] opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 46.1 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

Next, Alfred has already created an account on Langfuse and has his API keys ready. If you haven’t done so yet, you can sign up for Langfuse Cloud [here](https://cloud.langfuse.com/) or explore [alternatives](https://huggingface.co/docs/smolagents/tutorials/inspect_runs).  

Once you have your API keys, they need to be properly configured as follows:

In [29]:
import os
import base64
# from google.colab import userdata

LANGFUSE_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY=os.getenv("LANGFUSE_SECRET_KEY")

#LANGFUSE_PUBLIC_KEY=userdata.get("LANGFUSE_PUBLIC_KEY")
#LANGFUSE_SECRET_KEY=userdata.get("LANGFUSE_SECRET_KEY")
LANGFUSE_AUTH=base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU data region
# os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

Finally, Alfred is ready to initialize the `SmolagentsInstrumentor` and start tracking his agent's performance.  

In [30]:
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

Alfred is now connected 🔌! The runs from `smolagents` are being logged in Langfuse, giving him full visibility into the agent's behavior. With this setup, he's ready to revisit previous runs and refine his Party Preparator Agent even further.  

In [31]:
from smolagents import CodeAgent, InferenceClientModel, LiteLLMModel

model = LiteLLMModel(
        model_id="ollama_chat/qwen2:7b",  # Or try other Ollama-supported models
        api_base="http://127.0.0.1:11434",  # Default Ollama local server
        num_ctx=8192,
)

agent = CodeAgent(tools=[], model=model)
alfred_agent = agent.from_hub('Gaabin/AlfredAgent', trust_remote_code=True)
alfred_agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme        │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2:7b ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/home/gan/venv/lib/python3.12/site-packages/smolagents/models.py:315: FutureWarning: Attribute last_input_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.input_tokens instead.
  warnings.warn(
/home/gan/venv/lib/python3.12/site-packages/smolagents/models.py:324: FutureWarning: Attribute last_output_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.output_tokens instead.
  warnings.warn(
Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}


─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  playlist = web_search(query="best villain themed songs")                                                         
  print(playlist)                                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}


Out - Final answer: ['Tie Your Mother Down - Queen', 'Living After Midnight - Judas Priest', 'You Shook Me All 
Night Long - AC/DC', 'Under Pressure - David Bowie & Freddie Mercury', 'I Wanna Be Sedated - Ramones', 'Pumped Up 
Kicks - Foster the People', 'Psycho Killer - Talking Heads']

[Step 5: Duration 2.14 seconds| Input tokens: 13,651 | Output tokens: 596]

Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}


['Tie Your Mother Down - Queen',
 'Living After Midnight - Judas Priest',
 'You Shook Me All Night Long - AC/DC',
 'Under Pressure - David Bowie & Freddie Mercury',
 'I Wanna Be Sedated - Ramones',
 'Pumped Up Kicks - Foster the People',
 'Psycho Killer - Talking Heads']

Alfred can now access this logs [here](https://cloud.langfuse.com/project/cm7bq0abj025rad078ak3luwi/traces/995fc019255528e4f48cf6770b0ce27b?timestamp=2025-02-19T10%3A28%3A36.929Z) to review and analyze them.  

Meanwhile, the [suggested playlist](https://open.spotify.com/playlist/0gZMMHjuxMrrybQ7wTMTpw) sets the perfect vibe for the party preparations. Cool, right? 🎶
